# Imports

In [ ]:
import cudf as pd
import cupy
import dask
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
import random
import shap
import seaborn as sns
import tensorflow as tf

from cuml import RandomForestRegressor as CudaRandomForest
from cuml.metrics import mean_absolute_error
from shap.plots import colors
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from xgboost import XGBRegressor

SEED = 100


def reset_seed(rnd_seed=SEED):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(rnd_seed)
    np.random.seed(rnd_seed)
    cupy.random.seed(rnd_seed)
    tf.random.set_seed(rnd_seed)


reset_seed()
dask.config.set(scheduler="threads", num_workers=30)


# Dataset
## Load

In [ ]:
df = pd.read_csv('dataset - Palmas/electricity.csv', sep=";", decimal=",", header=0)
df_climatic = pd.read_csv('dataset - Palmas/climatic.csv', sep=";", decimal=",", header=0)

df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df_climatic["date"] = pd.to_datetime(df_climatic["date"], format="%d/%m/%Y")

df.drop("order", axis=1, inplace=True)
df.set_index("date", inplace=True)
df_climatic.set_index("date", inplace=True)

plt.figure(figsize=(18, 5))
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams.update({'font.size': 20})
plt.plot(df["consumption"], label="IFPR - Palmas Campus", color="blue")
plt.xlabel('Year')
plt.ylabel('Consumption (KWh)')

ax = plt.gca()
ax.set_facecolor('white')
plt.grid(True, color='grey', linestyle="--", linewidth=0.75)
plt.legend(facecolor='white')
plt.savefig("results - Palmas 12m/Serie-Palmas.png", bbox_inches='tight')
plt.show()

df


## Autocorrelation

In [ ]:
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams.update({'font.size': 15})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=["blue"])

fig, ax = plt.subplots(figsize=(6, 3))
ax.set_facecolor('white')
plt.grid(True, color='grey', linestyle="--", linewidth=0.75)

plot_acf(df[:50]["consumption"].to_pandas(), ax=ax, lags=12)
plt.savefig("results - Palmas 12m/CORR-ACF-Palmas.png", bbox_inches='tight')
plt.show()


In [ ]:
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams.update({'font.size': 15})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=["blue"])

fig, ax = plt.subplots(figsize=(6, 3))
ax.set_facecolor('white')
plt.grid(True, color='grey', linestyle="--", linewidth=0.75)

plot_pacf(df[:50]["consumption"].to_pandas(), ax=ax, lags=12)
plt.savefig("results - Palmas 12m/CORR-PACF-Palmas.png", bbox_inches='tight')
plt.show()


## Hypotesis Tests

The hypothesis tests were carried out in the R language, according to the "Hypotesis Tests.R" File.

# Preprocessing
## Missing Values

In [ ]:
for index, row in df_climatic[df_climatic.isnull()].to_pandas().iterrows():
    df_mes = df_climatic[df_climatic["month"] == df_climatic.at[index, "month"]]
    for col in row.index:
        if pandas.isnull(df_climatic.at[index, col]):
            df_mes.at[index, col] = df_mes[col].sum() / df_mes[col][df_mes[col].isnull() == False].count()
            df_climatic.at[index, col] = df_mes.at[index, col]


## LAG Creation

In [ ]:
for lag_col in ["consumption"]:
    for i in range(1, 12 + 1):
        lag_eletricity = df[lag_col].shift(i)
        df[f'LAG_' + '{:02d}'.format(i)] = lag_eletricity


## Datasets Merge

In [ ]:
df = pd.merge(left=df, right=df_climatic, on=["date", "month", "year"], how="left").sort_index()
df

## Dummy Variables Creation

In [ ]:
df_meses = pd.get_dummies(df["month"].astype(int), prefix="", prefix_sep="", dtype=int).rename(
    columns={"1": "month_JAN", "2": "month_FEV", "3": "month_MAR", "4": "month_ABR", "5": "month_MAI", "6": "month_JUN",
             "7": "month_JUL", "8": "month_AGO", "9": "month_SET", "10": "month_OUT", "11": "month_NOV",
             "12": "month_DEZ"}
)
df_anos = pd.get_dummies(df["year"].astype(int), prefix="", prefix_sep="", dtype=int).rename(
    columns={"2017": "year_2017", "2018": "year_2018", "2019": "year_2019", "2020": "year_2020", "2021": "year_2021",
             "2022": "year_2022", "2023": "year_2023", "2024": "year_2024"}
)
df = pd.concat([df, df_meses, df_anos], axis=1)
df = df.drop(["month", "year"], axis=1)
df = df.astype("float32").dropna()

df_show = df.to_pandas()
df_show

# Correlation Analysis


### LAGS

In [ ]:
columns = df.to_pandas().filter(like="LAG_").columns.tolist()
columns.insert(0, "consumption")

corr_matrix = df[columns].dropna().to_pandas().corr(
    numeric_only=True)

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(corr_matrix,
            cmap="coolwarm",
            center=0,
            annot=True,
            fmt='.1g',
            ax=ax)
plt.show()

### Climatic and COVID Variables

In [ ]:
corr_matrix = df.drop(df.to_pandas().filter(like="LAG_").columns,
                      axis=1).drop(df.to_pandas().filter(like="month_").columns,
                                   axis=1).drop(df.to_pandas().filter(like="year_").columns,
                                                axis=1).dropna().to_pandas().corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(corr_matrix,
            cmap="coolwarm",
            center=0,
            annot=True,
            fmt='.1g',
            ax=ax)
plt.show()


# SHAP values Analysis


### Random Forest

In [ ]:
df_copy = df.copy().to_pandas()

x_electricity = df_copy.drop("consumption", axis=1)
y_electricity = df_copy["consumption"]
model_rf = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=SEED)
shap.initjs()

model_rf.fit(x_electricity, y_electricity)

explainer_rf = shap.Explainer(model_rf)
shap_rf = explainer_rf(x_electricity)

importance_rf = pandas.DataFrame(list(zip(x_electricity.columns, np.abs(shap_rf.values).mean(0))),
                                 columns=["feature", "rf importance"])
importance_rf = importance_rf.sort_values(by=["rf importance"])
importance_rf

In [ ]:
plt.figure(figsize=(12, 6))
colors.blue_rgb = "blue"
shap.plots.waterfall(shap_rf[-1], max_display=10, show=False)
plt.gcf().set_size_inches(12, 6)


for ax in plt.gcf().get_axes():
    ax.set_xlim(14500, 17500)
    
plt.tight_layout()
plt.savefig("results - Palmas 12m/Feature Select - SHAP WATERFALL RF.png", bbox_inches='tight')
plt.show()

In [ ]:
shap.plots.bar(shap_rf)


### XGBoost

In [ ]:
df_copy = df.copy().to_pandas()

x_electricity = df_copy.drop("consumption", axis=1)
y_electricity = df_copy["consumption"]

model_xgb = XGBRegressor(booster="gbtree", objective='reg:squarederror', random_state=SEED)
shap.initjs()

model_xgb.fit(x_electricity, y_electricity)

explainer_xgb = shap.Explainer(model_xgb)
shap_xgb = explainer_xgb(x_electricity)

importance_xgb = pandas.DataFrame(list(zip(x_electricity.columns, np.abs(shap_xgb.values).mean(0))),
                                  columns=["feature", "xgb importance"])
importance_xgb = importance_xgb.sort_values(by=["xgb importance"])
importance_xgb

In [ ]:
plt.figure(figsize=(12, 6))
shap.plots.waterfall(shap_xgb[0], max_display=10, show=False)
plt.gcf().set_size_inches(12, 6)

for ax in plt.gcf().get_axes():
    ax.set_xlim(14000, 22000)

plt.tight_layout()
plt.savefig("results - Palmas 12m/Feature Select - SHAP WATERFALL XGB.png", bbox_inches='tight')
plt.show()


In [ ]:
shap.plots.bar(shap_xgb)


### Average - RF and XGB

In [ ]:
importance = pandas.DataFrame(list(zip(x_electricity.columns, (
        np.abs(shap_rf.values).mean(0) + np.abs(shap_xgb.values).mean(0)) / 2)),
                              columns=["feature", "Mean RF/XGB importance"])

importance = importance.sort_values(by=["Mean RF/XGB importance"])
importance


In [ ]:
importance = importance.sort_values(by=["Mean RF/XGB importance"], ascending=False)

bar_features = list(importance[0:10]["feature"])
bar_features.append(f"Sum of {len(importance[10:])} other features")
bar_importances_electr = list(importance[0:10]["Mean RF/XGB importance"])
bar_importances_electr.append(importance[10:]["Mean RF/XGB importance"].sum())

bar_features = bar_features[::-1]
bar_importances_electr = bar_importances_electr[::-1]

plt.figure(figsize=(12, 12))
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams.update({'font.size': 16})

bars = plt.barh(bar_features, bar_importances_electr, color=colors.red_rgb)

for bar, val in zip(bars, bar_importances_electr):
    plt.text(bar.get_width() + 5 if val < 300 else bar.get_width() -5, bar.get_y() + bar.get_height() / 2, f'+{val:.2f}',
             va='center', ha='left' if val < 300 else 'right', color=colors.red_rgb if val < 300 else "white")

plt.savefig("results - Palmas 12m/Importance RF XGB.png", bbox_inches='tight')
plt.show()

# Features Selection


In [ ]:
importance = importance.sort_values(by=["Mean RF/XGB importance"])


def ft_removal_rf(n_ft_removed, dataset):
    print("RF: " +n_ft_removed)
    df_selected = dataset[n_ft_removed:]["feature"]
    x = df[df_selected]
    y = df["consumption"]

    rf = CudaRandomForest(n_bins=x.shape[0], random_state=SEED)

    cvs_rf = []
    for i_train, i_test in TimeSeriesSplit(n_splits=5, test_size=1).split(x, y):
        x_train, x_test = x.iloc[i_train].to_cupy(), x.iloc[i_test].to_cupy()
        y_train, y_test = y.iloc[i_train].to_cupy(), y.iloc[i_test].to_cupy()

        rf.fit(x_train, y_train)
        cvs_rf.append(int(mean_absolute_error(y_test, rf.predict(x_test))))
    return int(np.array(cvs_rf).mean())


def ft_removal_xgb(n_ft_removed, dataset):
    print("XGB: " +n_ft_removed)
    df_selected = dataset[n_ft_removed:]["feature"]

    x = df[df_selected]
    y = df["consumption"]

    xgb = XGBRegressor(device="cuda", random_state=SEED)

    cvs_xgb = []
    for i_train, i_test in TimeSeriesSplit(n_splits=5, test_size=1).split(x, y):
        x_train, x_test = x.iloc[i_train].to_cupy(), x.iloc[i_test].to_cupy()
        y_train, y_test = y.iloc[i_train].to_cupy(), y.iloc[i_test].to_cupy()

        xgb.fit(x_train, y_train)
        cvs_xgb.append(int(mean_absolute_error(y_test, xgb.predict(x_test))))

    return int(np.array(cvs_xgb).mean())


importance = importance.sort_values(by=["Mean RF/XGB importance"])

ft_rm_rf = dask.compute(
    [dask.delayed(ft_removal_rf)(i, importance) for i in range(importance["feature"].shape[0])])[0]
ft_rm_xgb = dask.compute(
    [dask.delayed(ft_removal_xgb)(i, importance) for i in range(importance["feature"].shape[0])])[0]


In [ ]:
ft_rm_mean = (np.array(ft_rm_rf) + np.array(ft_rm_xgb)) / 2
min_index = np.argmin(ft_rm_mean[:40])

plt.figure(figsize=(12, 6))
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams.update({'font.size': 16})
plt.plot([x for x in ft_rm_rf[:40]], label="RF", color=colors.red_rgb)
plt.plot([x for x in ft_rm_xgb[:40]], label="XGBoost", color="blue")
plt.plot([x for x in ft_rm_mean[:40]], label="Average - RF/XGBoost", color="black")
plt.scatter(min_index, ft_rm_mean[min_index], color='black', zorder=2)
plt.annotate(f"({min_index:.2f}, {ft_rm_mean[min_index]:.2f})", xy=(min_index, ft_rm_mean[min_index]),
             xytext=(-60, -25), textcoords='offset points', fontsize=16)

plt.xlabel('Number of features removed')
plt.ylabel('Mean Absolute Error')
ax = plt.gca()
ax.set_facecolor('white')
plt.grid(True, color='grey', linestyle="--", linewidth=0.5)
plt.legend(facecolor='white')
plt.savefig("results - Palmas 12m/Feature Removal.png", bbox_inches='tight')
plt.show()
importance = importance.sort_values(by=["Mean RF/XGB importance"])
df_selected = df.drop(importance[:min_index]["feature"], axis=1)
df_selected.to_pandas().to_csv(f"dataset - Palmas/elect_merged_selected.csv", sep=";", decimal=",")
df_selected